In [2]:
import os
import numpy as np
from data_generator import DataGenerator

In [10]:
label_name = {}
for index, folder in enumerate(os.listdir(f'data/train')):
    label_name[folder] = index-1

In [11]:
train_paths = np.array([])
train_labels = {}
for folder in os.listdir('data/train'):
    for file in os.listdir(f'data/train/{folder}'):
        train_paths = np.append(train_paths, np.array([f'data/train/{folder}/{file}']))
        train_labels[f'data/train/{folder}/{file}'] = label_name[folder]

validate_paths = np.array([])
validate_labels = {}
for folder in os.listdir('data/validation'):
    for file in os.listdir(f'data/validation/{folder}'):
        validate_paths = np.append(validate_paths, np.array([f'data/validation/{folder}/{file}']))
        validate_labels[f'data/validation/{folder}/{file}'] = label_name[folder]

In [17]:
train_generator = DataGenerator(train_paths, train_labels, batch_size=128)
validate_generator = DataGenerator(validate_paths, validate_labels, batch_size=128)

In [24]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Dense, GlobalMaxPooling2D

model = Sequential([
    Conv2D(filters=16, kernel_size=(3,3), input_shape=(250,250,3), activation='relu', padding='same'),
    Conv2D(filters=1, kernel_size=(2,2), strides=(2,2)),
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    GlobalMaxPooling2D(),
    Dense(5, activation='softmax')
    ])

In [25]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics='accuracy')

In [26]:
os.makedirs('models')

FileExistsError: [Errno 17] File exists: 'models'

In [27]:
import datetime
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
now = datetime.datetime.today()
callback = [ModelCheckpoint(f'model/{now}.save', monitor='val_loss', verbose=1, save_best_only=True), EarlyStopping(monitor='val_loss', verbose=1, patience=10)]

In [ ]:
model.fit(train_generator, epochs=100, batch_size=128, validation_data=validate_generator, callbacks=callback)

Epoch 1/100
311/468 [==================>...........] - ETA: 22:46 - loss: 0.9962 - accuracy: 0.5980